In [5]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.3.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Reading package lists... Done


In [6]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2022-10-14 23:18:34--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  6.19MB/s    in 0.2s    

2022-10-14 23:18:35 (6.19 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [7]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-ChallengeD2").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [8]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Books_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|        review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-------------------+
|         US|   25933450| RJOVP071AVAJO|0439873800|      84656342|There Was an Old ...|           Books|          5|            0|          0|   N|                Y|          Five Stars|I love it and so ...|2015-08-31 00:00:00|
|         US|    1801372|R1ORGBETCDW3AI|1623953553|     729938122|      I Saw a Friend| 

In [9]:
# Create the vine_table. DataFrame
vine_df = df.withColumn("star_rating",df["star_rating"].cast("int")).select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RJOVP071AVAJO|          5|            0|          0|   N|                Y|
|R1ORGBETCDW3AI|          5|            0|          0|   N|                Y|
| R7TNRFQAOUTX5|          5|            0|          0|   N|                Y|
|R2GANXKDIFZ6OI|          5|            0|          0|   N|                N|
|R2NYB6C3R8LVN6|          5|            2|          2|   N|                Y|
|R13U5PBJI1H94K|          2|            1|          1|   N|                N|
|R1H8UVH990F8VE|          5|            2|          2|   N|                N|
|R2MC0N30WZMRQ5|          5|            0|          0|   N|                Y|
| R2NO2HXK16Y4J|          5|            0|          0|   N|                Y|
|R245YIAVJK82ZL|          5|            0|          0|   N|     

In [13]:
vine_df_total_votes = vine_df.filter(vine_df["total_votes"] >= 20)
vine_df_total_votes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|
|R2QAPWF1M5863L|          1|            5|         23|   N|                N|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|
|R28WZIA84DUP6C|          2|            4|         43|   N|                N|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|
|R1XVYF74WCJKO3|          1|           47|         49|   N|                N|
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|
|R3KK9MHFLY2BVW|          5|           21|         22|   N|     

In [15]:
vine_df_helpful = vine_df_total_votes.filter(vine_df_total_votes["helpful_votes"] / vine_df_total_votes["total_votes"] >= .5 )
vine_df_helpful.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|
|R1XVYF74WCJKO3|          1|           47|         49|   N|                N|
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|
|R3KK9MHFLY2BVW|          5|           21|         22|   N|                N|
|R2OEP8MGX5K370|          5|           30|         34|   N|                Y|
| RTX63ENBKWCXF|          1|           28|         35|   N|     

In [16]:
vine_df_vine = vine_df_helpful.filter(vine_df_helpful["vine"] == "Y")
vine_df_vine.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3KKWNG5VABDYC|          5|           31|         32|   Y|                N|
|R397PIJYYVJ7PD|          4|           31|         32|   Y|                N|
| RP6Y2CTIFUYD0|          2|           31|         32|   Y|                N|
|R1FYRNM5U0S2VZ|          3|           20|         24|   Y|                N|
| R1ME19UY4UNAO|          5|           53|         57|   Y|                N|
|R1JW7ZHT2VLT1E|          3|           40|         45|   Y|                N|
| R98UYXY43TURV|          4|           46|         50|   Y|                N|
|R34VXM3EJDS78Z|          3|          374|        410|   Y|                N|
|R1QX8UOW5JO6L0|          4|           20|         21|   Y|                N|
| R5Y5SXCRQ2GNH|          3|           34|         37|   Y|     

In [20]:
vine_df_vine.count()

5012

In [17]:
vine_df_not_vine = vine_df_helpful.filter(vine_df_helpful["vine"] == "N")
vine_df_not_vine.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3T6LR4H11XRAN|          5|           26|         26|   N|                Y|
|R3QFNFMFWOQIBO|          3|           15|         25|   N|                Y|
|R24S2SV0ZIFIMV|          1|           41|         78|   N|                N|
|R3OCONKYE1A047|          5|           33|         38|   N|                N|
|R2U7YNJZ5JHWVD|          5|           23|         25|   N|                Y|
|R1XVYF74WCJKO3|          1|           47|         49|   N|                N|
|R3KK9MHFLY2BVW|          5|           21|         22|   N|                N|
|R2OEP8MGX5K370|          5|           30|         34|   N|                Y|
| RTX63ENBKWCXF|          1|           28|         35|   N|                N|
|R38I0D0J8O6LB6|          5|          101|        102|   N|     

In [23]:
totals_df = spark.createDataFrame([
    ("Paid", vine_df_vine.count()),
    ("Unpaid", vine_df_not_vine.count())
], ["Review Type", "Total"])

totals_df.show()

+-----------+------+
|Review Type| Total|
+-----------+------+
|       Paid|  5012|
|     Unpaid|109297|
+-----------+------+



In [31]:
five_df = spark.createDataFrame([
    ("Paid", vine_df_vine.filter(vine_df_vine["star_rating"] == 5).count()),
    ("Unpaid", vine_df_not_vine.filter(vine_df_not_vine["star_rating"] == 5).count())
], ["Review Type", "Number of 5-Star Reviews"])

five_df.show()

+-----------+------------------------+
|Review Type|Number of 5-Star Reviews|
+-----------+------------------------+
|       Paid|                    2031|
|     Unpaid|                   49967|
+-----------+------------------------+



In [32]:
five_per_df = spark.createDataFrame([
    ("Paid", vine_df_vine.filter(vine_df_vine["star_rating"] == 5).count() / vine_df_vine.count()),
    ("Unpaid", vine_df_not_vine.filter(vine_df_not_vine["star_rating"] == 5).count() / vine_df_not_vine.count())
], ["Review Type", "Percentage of 5-Star Reviews"])

five_per_df.show()

+-----------+----------------------------+
|Review Type|Percentage of 5-Star Reviews|
+-----------+----------------------------+
|       Paid|         0.40522745411013567|
|     Unpaid|         0.45716716835777743|
+-----------+----------------------------+



In [33]:
summary_df = spark.createDataFrame([
    ("Paid", vine_df_vine.count(), vine_df_vine.filter(vine_df_vine["star_rating"] == 5).count(), vine_df_vine.filter(vine_df_vine["star_rating"] == 5).count() / vine_df_vine.count()),
    ("Unpaid", vine_df_not_vine.count(), vine_df_not_vine.filter(vine_df_not_vine["star_rating"] == 5).count(), vine_df_not_vine.filter(vine_df_not_vine["star_rating"] == 5).count() / vine_df_not_vine.count())
], ["Review Type", "Total", "Number of 5-Star Reviews", "Percentage of 5-Star Reviews"])

summary_df.show()

+-----------+------+------------------------+----------------------------+
|Review Type| Total|Number of 5-Star Reviews|Percentage of 5-Star Reviews|
+-----------+------+------------------------+----------------------------+
|       Paid|  5012|                    2031|         0.40522745411013567|
|     Unpaid|109297|                   49967|         0.45716716835777743|
+-----------+------+------------------------+----------------------------+

